In [155]:
import json
import pandas as pd

In [156]:
# открываем нужные коллекции
jobs_file = '/Users/vadim/Downloads/imarketplacedevelopment/jobs.json'
offers_file = '/Users/vadim/Downloads/imarketplacedevelopment/offers.json'
specialists_file = '/Users/vadim/Downloads/imarketplacedevelopment/specialists.json'
skills_file = '/Users/vadim/Downloads/imarketplacedevelopment/skills.json'
offerstatuses_file = '/Users/vadim/Downloads/imarketplacedevelopment/offerstatuses.json'
with open(jobs_file) as train_file: 
    dict_jobs = json.load(train_file)
with open(offers_file) as train_file: 
    dict_offers = json.load(train_file)
with open(specialists_file) as train_file: 
    dict_specs = json.load(train_file)
with open(skills_file) as train_file: 
    dict_skills = json.load(train_file)
with open(offerstatuses_file) as train_file: 
    dict_offstatuses = json.load(train_file)

In [157]:
# делаем датафреймы из коллекций json
df_offers = pd.json_normalize(dict_offers)
df_specs = pd.json_normalize(dict_specs)
df_skills = pd.json_normalize(dict_skills)
df_offstatuses = pd.json_normalize(dict_offstatuses)

In [158]:
# составляем список из расшифровок статусов по айдишнику
offstatuses = []
for status in df_offers['status.$oid']:
    statuses = df_offstatuses.loc[df_offstatuses['_id.$oid'] == status]
    offstatuses.append(statuses['name.ru'].iloc[0])

In [159]:
# составляем словарь (ключ - айдишник вакансии, значение - требования)
job_reqs={}
for job in dict_jobs:
    job_req = ''
    for requirement in job['requirements']:
        job_req += requirement['requirement']['ru']
    job_reqs[job['_id']['$oid']] = job_req

In [160]:
# составляем список из требований по айдишнику вакансии
requierements = []
for job in df_offers['job.$oid']:
    requierements.append(job_reqs[job])

In [161]:
# составляем список из скиллов специалиста
spec_skills = []
for skills in df_specs['skills']:
    tmp_skills = ''
    for skill in skills:
        tmp_skill = df_skills.loc[df_skills['_id.$oid'] == skill]
        tmp_skills += tmp_skill['name.ru'].iloc[0]
        tmp_skills += ';'
    spec_skills.append(tmp_skills)
    
df_specs['spec_skills'] = spec_skills

In [162]:
# составляем список из скиллов спеца по его айдишнику
skills = []
for spec in df_offers['specialist.$oid']:
    tmp_skills = df_specs.loc[df_specs['_id.$oid'] == spec]
    if tmp_skills.empty:
        skills.append(None)
    else:
        skills.append(tmp_skills['spec_skills'].iloc[0])

In [163]:
# добавляем расшифровки требований
df_offers = df_offers.drop('job.$oid', axis=1)
df_offers['requirements'] = requierements

In [164]:
# добавляем расшифровки статусов
df_offers = df_offers.drop('status.$oid', axis=1)
df_offers['status'] = offstatuses

In [165]:
# добавляем расшифровки скиллов
df_offers = df_offers.drop('specialist.$oid', axis=1)
df_offers['skills'] = skills

In [166]:
# удаляем ненужный столбец
df_offers = df_offers.drop('__v', axis=1)

In [167]:
df_offers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   rate             914 non-null    int64 
 1   requirements     914 non-null    object
 2   _id.$oid         914 non-null    object
 3   createdAt.$date  914 non-null    object
 4   updatedAt.$date  914 non-null    object
 5   status           914 non-null    object
 6   skills           464 non-null    object
dtypes: int64(1), object(6)
memory usage: 50.1+ KB


In [168]:
# удаляем строки с пустыми значениями
df_offers.dropna()

,rate,requirements,_id.$oid,createdAt.$date,updatedAt.$date,status,skills
18,150000,Не менее 3 лет опыта работы в роли ML Engineer...,673629b8cd51283bbb3d9909,2024-11-14T16:47:52.973Z,2024-11-14T16:47:52.973Z,Новый,Git;Python;Linux;Ubuntu;C++;C#;PHP;Laravel;HTML;
23,0,Не менее 3 лет опыта работы в роли ML Engineer...,673629b8cd51283bbb3d9915,2024-11-14T16:47:52.985Z,2024-11-14T16:47:52.985Z,Новый,C++;Python;SQL;ООП;PostgreSQL;NLP;Английский я...
31,0,Не менее 3 лет опыта работы в роли ML Engineer...,673629b9cd51283bbb3d9923,2024-11-14T16:47:53.005Z,2024-11-14T16:47:53.005Z,Новый,Аналитическое мышление;Python;SQL;NoSQL;ML;Мат...
34,0,Не менее 3 лет опыта работы в роли ML Engineer...,673629b9cd51283bbb3d9970,2024-11-14T16:47:53.225Z,2024-11-14T16:47:53.225Z,Новый,Английский язык;Python;SQL;PostgreSQL;Keras;
208,0,"Наличие опыта тестировщика от 2,5 лет;Опыт раб...",673636fccd51283bbb3dbde9,2024-11-14T17:44:28.802Z,2024-11-14T17:44:28.802Z,Новый,Управление командой;Аналитическое мышление;
...,...,...,...,...,...,...,...
909,135000,"Наличие опыта тестировщика от 2,5 лет;Опыт раб...",673686cc76597ebe1ac2aa27,2024-11-14T23:25:00.554Z,2024-11-14T23:25:00.554Z,Новый,Функциональное тестирование;Ручное тестировани...
910,0,"Наличие опыта тестировщика от 2,5 лет;Опыт раб...",673686cc76597ebe1ac2aa29,2024-11-14T23:25:00.557Z,2024-11-14T23:25:00.557Z,Новый,Linux;SQL;CSS;Функциональное тестирование;Тест...
911,0,"Наличие опыта тестировщика от 2,5 лет;Опыт раб...",673686e276597ebe1ac2abb1,2024-11-14T23:25:22.428Z,2024-11-14T23:25:22.428Z,Новый,SQL;Функциональное тестирование;API;Postman;Gi...
912,0,"Наличие опыта тестировщика от 2,5 лет;Опыт раб...",673686e276597ebe1ac2abb4,2024-11-14T23:25:22.435Z,2024-11-14T23:25:22.435Z,Новый,SQL;Git;MySQL;Linux;PostgreSQL;REST API;JSON;H...
